In [1]:
import os
import requests
from bs4 import BeautifulSoup
import time

In [2]:
urls = [
  {'url':'https://www.yodobashi.com/product/100000001007205381/', 'name':'yodobashi'},
  {'url':'https://www.yodobashi.com/product/100000001003431572/', 'name':'yodobashi'},
  {'url':'https://ymall.jp/kaden/5712100017/', 'name':'yamada'},
  {'url': 'https://ymall.jp/kaden/1519200011/', 'name':'yamada'},
]

In [3]:
def get_soup(url):
  """
  urlからBeautifulSoupオブジェクトを取得する
  
  Parameters
  ----------
  url : str
    対象のURL
      
  Returns
  -------
  soup : BeautifulSoup
    BeautifulSoupオブジェクト
  """
  time.sleep(1)
  res = requests.get(url)
  return BeautifulSoup(res.text, 'html.parser')

In [4]:
def yodobashi_crowl(url):
  """
  ヨドバシのクローリングを行う
  """
  soup = get_soup(url)
  print(soup.select('.buyBoxMain'))
  

In [5]:
yodobashi_crowl(urls[0]['url'])

In [15]:
def main():
    send_line_notify('てすとてすと')

def send_line_notify(notification_message):
    """
    LINEに通知する

    環境変数 YOUR_LINE_NOTIFY_TOKEN にトークンを設定しておくこと

    Parameters
    ----------
    notification_message : str
        通知するメッセージ

    Returns
    -------
    None
    """
    line_notify_token = os.environ['YOUR_NOTIFY_TOKEN']
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)

if __name__ == "__main__":
    main()